<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

## Ion Selection and Rangging

This tutorial outlines a comprehensive workflow for ion selection and organization. Users can choose ions using peak and element finders, manually add ions, customize ion colors, and create histograms. Histograms can be generated for selected ranges and areas, and figures can be saved. Additionally, users have the option to save both figures and data in CSV and HDF5 formats.

</div>

In [1]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import pandas as pd
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import Output
from IPython.display import clear_output
import matplotlib.colors as mcolors
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import share_variables, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, dataset_path_qt
from pyccapt.calibration.calibration_tools import mc_plot

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

By clicking on the button below, you can select the dataset file you want to use. The dataset file can be in various formats, including HDF5, EPOS, POS, ATO, and CSV.
</div>

In [15]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">
From the dropdown lists below, you can select the instrument specifications of the dataset. The TDC model and flight path distance are required for the reconstruction.
</div>


In [16]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, flightPathLength_d)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [11]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/ions_selection/'
variables.result_data_name = 'range_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

NameError: name 'dataset_path' is not defined

In [18]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/ions_selection/
The dataset name after saving is: range_pyccapt_data_1642_Aug-30-2023_16-05_Al_test4
The figures will be saved on the path: D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4//ions_selection/
Total number of Ions: 10559766


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,21.387474,4.793744,3.742590,27.362214,28.993453,5019.959961,1003.992004,7565,572.395831,618.564148,2.651428,0.594286,0,0
1,-0.465509,5.449197,0.226819,27.034338,29.360520,5019.959961,1003.992004,7664,569.176119,606.260925,-0.055510,0.649796,99,2
2,-0.903869,5.094536,0.202985,27.100974,29.441372,5019.959961,1003.992004,7682,569.832045,606.932983,-0.107755,0.607347,18,1
3,18.634826,-3.726965,2.792876,27.100149,28.725689,5019.959961,1003.992004,7690,569.823932,611.520996,2.285714,-0.457143,8,1
4,4.014661,14.361170,1.705268,27.236032,29.512555,5019.959961,1003.992004,7794,571.159024,614.298462,0.486531,1.740408,104,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10559761,-4.831920,15.283945,57.205555,27.244832,29.759238,6351.169922,1270.234009,19549,571.245368,550.999146,-0.460408,1.456326,105,1
10559762,-14.125940,-1.271678,56.866164,27.414528,29.967560,6351.169922,1270.234009,19664,572.907770,551.705505,-1.342041,-0.120816,31,1
10559763,16.429191,3.148928,57.345214,27.119032,29.029561,6351.169922,1270.234009,19762,570.009661,545.156128,1.567347,0.300408,98,1
10559764,13.831981,-9.107195,57.311214,26.980992,28.815053,6351.169922,1270.234009,19833,568.650437,543.187866,1.319184,-0.868571,71,1


<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

The calibrated mass-to-charge ratio that is calculated with previous workflow will be shown below. You can select the range of the data you want to plot. There is also possibility to activate the peak finding tool to show the peak values.
</div>

In [19]:
interact_manual(mc_plot.hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), log=widgets.Dropdown(options=[('True', True), ('Flase', False)]),
                target=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), mode=widgets.Dropdown(options=[('normal', 'normal'), ('normalized', 'normalized')]),
                prominence=widgets.IntText(value=100), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=variables.max_tof), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]), range_plot=fixed(False), selected_area=fixed(False), print_info=fixed(True));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='log', options=(('Tru…

In [2]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=50, description='peak prominance:')
distance=widgets.IntText(value=1, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot_p(variables):
    
    with out:
        clear_output(True)
        # clear the peak_idx
        variables.peaks_idx = []
        mc_plot.hist_plot(variables, bin_size.value, log=True, target='mc_c', mode='normal', prominence=prominence.value, distance=distance.value, percent=percent.value,
                          selector='peak', figname=index_fig.value, lim=lim_tof.value, peaks_find_plot=plot_peak.value, print_info=False)

def hist_plot_r(variables):
    
    with out:
        clear_output(True)
        print('=============================')
        print('Press left click to draw a line')
        print('Press right click to remove a line')
        print('Press r to remove all the line')
        print('Hold shift and use mouse scroll for zooming on x axis')
        print('Hold ctrl and left mouse bottom to move a line')
        print('=============================')
        mc_plot.hist_plot(variables, bin_size.value, log=True, target='mc_c', mode='normal', prominence=prominence.value, distance=distance.value, percent=percent.value,
                          selector='range', figname=index_fig.value, lim=lim_tof.value, peaks_find_plot=True, ranging_mode=True, save_fig=False, print_info=False)


In [7]:
# element calculate
peak_val = widgets.FloatText(value=1.1, description='peak value:')

mass_difference = widgets.FloatText(value=2, description='mass range:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)
num_element = widgets.IntText(value=5, description='num element:')
# formula calculate
formula_m = widgets.Text(
    value='{12}C1{16}O2',
    placeholder='Type a formula  {12}C1{16}O2',
    description='Isotope formula:',
    disabled=False
)

molecule_charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

# molecule create
formula_com = widgets.Text(
    value='',
    placeholder="H, O",
    description='Elements:',
    disabled=False
)
complexity = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='complexity:'
)

charge_com = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

In [25]:
plot_button_p = widgets.Button(
    description='plot hist',
)

plot_button_r = widgets.Button(
    description='plot hist',
)

plot_button = widgets.Button(
    description='plot hist',
)


find_elem_button = widgets.Button(
    description='find element',
)

plot_element = widgets.Button(
    description='plot element',
)

formula_button = widgets.Button(
    description='manual formula',
)

add_ion_button = widgets.Button(
    description='add ion',
)
romove_ion_button = widgets.Button(
    description='remove ion',
)
show_color = widgets.Button(
    description='show color',
)
change_color = widgets.Button(
    description='change color',
)

color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
@plot_button_p.on_click
def plot_on_click_p(b, plot=True):
    hist_plot_p(variables)
    
@plot_button_r.on_click
def plot_on_click_r(b, plot=True):
    hist_plot_r(variables)
    
@plot_element.on_click  
def plot_finded_elemnt(b,):
    variables.AptHistPlotter.plot_founded_range_loc(variables.ions_list_data, remove_lines=False)
@find_elem_button.on_click
def vol_on_click(b,):
    with output2:
        clear_output(True)
        df1 = ion_selection.load_elements(formula_com.value, aboundance_threshold.value, charge.value, variables=variables)
        df2 = ion_selection.molecule_create(formula_com.value, complexity.value, charge.value, aboundance_threshold.value, variables)
        df3 = ion_selection.find_closest_elements(peak_val.value, num_element.value, aboundance_threshold.value, charge.value, variables=variables)
        df = pd.concat([df1, df2, df3], axis=0)
        df = df[(df['abundance'] >= aboundance_threshold.value)]
        df = df[abs(df['mass'] - peak_val.value) <= mass_difference.value]
        df = df.iloc[(df['mass'] - peak_val.value).abs().argsort()]
        # Reset the index to maintain a clean index order
        df.reset_index(drop=True, inplace=True)
        df = df[:num_element.value]
        variables.range_data_backup = df.copy()
        variables.ions_list_data = df.copy()
        display(df)
    
@formula_button.on_click
def manual_formula(b,):
    with output2:
        if formula_m.value == '':
            print("Input is empty. Type the formula.")
        else:
            df = ion_selection.molecule_manual(formula_m.value, molecule_charge.value, latex=True, variables=variables)
            clear_output(True)
            display(df)

@add_ion_button.on_click
def add_ion_to_range_dataset(b,):
    ion_selection.ranging_dataset_create(variables, row_index.value, peak_val.value)
    with output3:
        clear_output(True)
        display(variables.range_data)
@romove_ion_button.on_click
def remove_ion_to_range_dataset(b,):
    if len(variables.range_data) >= 1:
        variables.range_data = variables.range_data.drop(len(variables.range_data) - 1)
        with output3:
            clear_output(True)
            display(variables.range_data)
        
@show_color.on_click
def show_color_ions(b,):
    with output3:
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
@change_color.on_click
def change_color_m(b,):
    with output3:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

        # Create "Next" and "Previous" buttons
start_button = widgets.Button(description="start")
next_button = widgets.Button(description="next")
prev_button = widgets.Button(description="previous")
reset_zoom_button = widgets.Button(description="reset zoom")
all_peaks_button = widgets.Button(description="add all peaks")


# Define button click events
@start_button.on_click
def start_peak(_):
    variables.h_line_pos = []
    print('=============================')
    print('Press left click to draw a line')
    print('Press right click to remove a line')
    print('Press r to remove all the line')
    print('Press a to automatically draw lines')
    print('Hold shift and use mouse scroll for zooming on x axis')
    print('Hold ctrl and left mouse bottom to move a line')
    print('=============================')
    variables.peaks_index = 0
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value )
    variables.AptHistPlotter.zoom_to_x_range(x_min=peak_val.value - 5, x_max=peak_val.value + 5, reset=False)
@next_button.on_click
def next_peak(_):
    variables.peaks_index += 1
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value)
    variables.AptHistPlotter.zoom_to_x_range(x_min=peak_val.value - 5, x_max=peak_val.value + 5, reset=False)
@prev_button.on_click
def prev_peak(_):
    variables.peaks_index -= 1
    peak_val.value = variables.peaks_x_selected[variables.peaks_index]
    print('peak idc:', variables.peaks_index, 'Peak location:', peak_val.value )
    variables.AptHistPlotter.zoom_to_x_range(x_min=peak_val.value - 5, x_max=peak_val.value + 5, reset=False)

@reset_zoom_button.on_click
def rest_h_line(_):
    variables.AptHistPlotter.zoom_to_x_range(x_min=0, x_max=0, reset=True)
@all_peaks_button.on_click
def select_all_peaks(_):
    variables.peaks_idx = variables.peak_x
    
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_p, all_peaks_button])])
tab2 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, HBox(children=[VBox(children=[plot_button_r, start_button, next_button, prev_button, reset_zoom_button])])])
tab4 = VBox(children=[HBox(children=[VBox(children=[peak_val, charge, aboundance_threshold, mass_difference, num_element, formula_com, complexity, find_elem_button, plot_element]),
                                    VBox(children=[row_index, color_picker, add_ion_button, romove_ion_button, show_color, change_color]), 
                                    VBox(children=[formula_m, molecule_charge, formula_button])])])

tabs1 = widgets.Tab(children=[tab1, tab2])
tabs2 = widgets.Tab(children=[tab4])
tabs1.set_title(0, 'peak finder')
tabs1.set_title(1, 'ranging')
tabs2.set_title(0, 'element finder')
# Create two Output widgets to capture the output of each plot
out = Output()
output2 = Output()
output3 = Output()

# Create an HBox to display the buttons side by side
buttons_layout = widgets.HBox([tabs1, tabs2])

# Create a VBox to display the output widgets below the buttons
output_layout_h = widgets.HBox([out, output3])
output_layout = widgets.VBox([output_layout_h, output2])

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

To do the ranging first we wave to find and select the peaks by using the peak finder tab. Then in the ranging tab we can zoom the selected peak and specify the range by drawing two vertical line around the peak. For each peak by pressing the element find bottom we see the possible ions and moleculse that have same mass. It is possible to plot these ions and molecules by pressing the plot element bottom. After finding the element we can add the ions and related range by setting the index of it from the dropdown ad pressing the add bottom. We have to repast this steps for all selected peaks.

</div>

In [26]:
# Display the buttons and the output widgets
display(buttons_layout, output_layout)

with output3:
    display(variables.range_data)

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Here base on the rangged dataset we can plot the mass spectrum and see the selected ranges.

</div>

In [47]:
interact_manual(mc_plot.hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), log=widgets.Dropdown(options=[('True', True), ('False', False)]),
                target=widgets.Dropdown(options=[('mc_c', 'mc_c')]), mode=widgets.Dropdown(options=[('normal', 'normal'), ('normalized', 'normalized')]),
                prominence=widgets.IntText(value=100), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]), range_plot=widgets.Dropdown(options=[('True', True), ('False', False)]), selected_area=fixed(False), print_info=fixed(True)
               , ranging_mode=fixed(False), save_fig=fixed(True));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='log', options=(('Tru…

In [48]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.067500,0.904119,1.308847,#b2aa2d,['H'],[1],[1],1
1,${}^{27}Al^{2+}$,13.490000,14.478300,13.873421,15.381949,#e7e0d1,['Al'],[1],[27],2
2,${}^{27}Al^{+}$,26.980000,29.290200,27.888184,31.812810,#e7e0d1,['Al'],[1],[27],1


In [27]:
variables.range_data.dtypes

ion         object
mass       float64
mc         float64
mc_low     float64
mc_up      float64
color       object
element     object
complex     object
isotope     object
charge      uint32
dtype: object

<div style="margin: 0 auto; padding: 20px; font-family: 'Helvetica', sans-serif; font-size: 16px;">

Finaly we save the range in the hdf5 and csv format.

</div>

In [50]:
# save the new data
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)

In [51]:
# save data in csv format
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)